In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423
┌ Warning: Package BpAlignGpu does not have OffsetArrays in its dependencies:
│ - If you have BpAlignGpu checked out for development and have
│   added OffsetArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with BpAlignGpu
└ Loading OffsetArrays into BpAlignGpu from project dependency, future warnings for BpAlignGpu are suppressed.


In [2]:
T = Float32

Float32

In [3]:
#open("/home/louise/MSA/Data/test/PF00684/")
open("/home/louise/MSA/Data/test/PF00397/")
q = 21; L = 31; 
ctype=Symbol("amino")
typel=Symbol("bm")

#J, H = BpAlignGpu.read_parameters("/home/louise/MSA/Data/test/PF00684/Parameters_bm_PF00684seed_potts.dat", q, L, gap=0, typel=typel);
J, H = BpAlignGpu.read_parameters("/home/louise/MSA/Data/test/PF00397/Parameters_plm_PF00397.dat", q, L, gap=0, typel=typel);

┌ Info: Assuming J i j a b and h i a format
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:37
┌ Info: Output tersors: J[a b i j] and h[a i]
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:39
┌ Info: Gap in input file 0 now in 21
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:40


In [4]:
delta = 5;
#al = BpAlignGpu.enveloptoalign( "/home/louise/MSA/Data/test/PF00684/Test_PF00684.full", "/home/louise/MSA/Data/test/PF00684/Test_PF00684.fasta", "/home/louise/MSA/Data/test/PF00684/Test_PF00684.ins", delta = delta, ctype = ctype);
al = BpAlignGpu.enveloptoalign( "/home/louise/MSA/Data/test/PF00397/PF00397_full_length_sequences.fasta", "/home/louise/MSA/Data/test/PF00397/PF00397_full.fasta", "/home/louise/MSA/Data/test/PF00397/PF00397_full.inse", delta = delta, ctype = ctype);
M = length(al)

println("Consider this sequence: ")
# consider one random sequence as an example
idx0 = 20# rand(1:M) 
(aux,garb) = split(al[idx0][1], "/")
seqsol = al[idx0][3];
seqins = al[idx0][4];
seq = BpAlignGpu.Seq(seqsol, al[idx0][2], ctype)
N = length(al[idx0][2])
println(al[idx0][1])
println("Full length: ")
println(al[idx0][2])
println("Aligned by HMMer (without and with insertions): ")
println(seqsol)
println(seqins)



Consider this sequence: 
A0A401GBT9_9APHY/8-47
Full length: 
MAPPPLPLGWTEHMSPTGQPYYFNTQTQESTYVRPLPPFP
Aligned by HMMer (without and with insertions): 
LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP
LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP


In [5]:
using DelimitedFiles


In [6]:
#Lambda_all = readdlm("/home/louise/MSA/Data/test/PF00684/Lambda_PF00684.dat")
Lambda_all = readdlm("/home/louise/MSA/Data/test/PF00397/Lambda_PF00397.dat")
lambda_o = Lambda_all[1,:];
lambda_e = Lambda_all[2,:];


muext = 0.00;
muint = 2.50;

In [7]:
q, N, L

(21, 40, 31)

In [8]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=31 N=40 q=21]


In [9]:
damp=T(0.2)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.2
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------


In [10]:
bpm = BPMessages(seq, pm)
bpb = BPBeliefs(N, L)
lrf = LongRangeFields(N, L)
af = AllFields(bpm, bpb, lrf)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


AllFields{Float32}[L=31 N=40 ongpu=true]

In [27]:
@time BpAlignGpu.test_sweep!(50,af,pm,pa)

t=1	 err=0.1942451
t=2	 err=0.5556629
t=3	 err=0.747076
t=4	 err=0.007356763
t=5	 err=0.0072240233
t=6	 err=0.0073447227
t=7	 err=0.44053686
t=8	 err=0.4417041
t=9	 err=0.00727129
t=10	 err=0.00731349
t=11	 err=0.17186898
t=12	 err=0.1745404
t=13	 err=0.0072885156
t=14	 err=0.083426416
t=15	 err=0.08561933
t=16	 err=0.0072950125
t=17	 err=0.007293701
t=18	 err=0.007294297
t=19	 err=0.29226124
t=20	 err=0.29417372
t=21	 err=0.022338074
t=22	 err=0.02560383
t=23	 err=0.03159213
t=24	 err=0.035234928
t=25	 err=0.009208143
t=26	 err=0.007294059
t=27	 err=0.007294059
t=28	 err=0.0072941184
t=29	 err=0.054297265
t=30	 err=0.1575607
t=31	 err=0.19784462
t=32	 err=0.01349614
t=33	 err=0.0072941184
t=34	 err=0.007294059
t=35	 err=0.007294059
t=36	 err=0.40429223
t=37	 err=0.4063865
t=38	 err=0.008381665
t=39	 err=0.007294059
t=40	 err=0.25430498
t=41	 err=0.22610003
t=42	 err=0.06415677
t=43	 err=0.0881291
t=44	 err=0.05477637
t=45	 err=0.03474188
t=46	 err=0.014907777
t=47	 err=0.13804632
t=48

In [50]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


In [51]:
p0 = heatmap(af.bpb.beliefs[:,1, :], xlabel="i", ylabel="n", title="x=0")

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007fc2105b8460.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArrays /home/louise/.julia/packages/GPUArrays/umZob/src/host/indexing.jl:56
┌ Info: Precompiling GR_jll [d2c73de3-f751-5644-a686-071e5b155ba9]
└ @ Base loading.jl:1423


ErrorException: GPU broadcast resulted in non-concrete element type Any.
This probably means that the function you are broadcasting contains an error or type instability.

In [28]:
solbel, maxbel = BpAlignGpu.solmaxbel(af);
minimum(maxbel)

0.7974027f0

In [49]:
BpAlignGpu.convert_soltosequence!(solbel, seq.strseq, N, L)

("LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP", "LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP", 6, 35)

In [29]:
using OffsetArrays
P = fill(fill(0.0, 0:1,0:N+1), L)
for i=1:L
    P[i] = OffsetArray(transpose(af.bpb.beliefs[:,:,i]), 0:1,0:N+1)
end
P[2]

2×42 OffsetArray(::Matrix{Float64}, 0:1, 0:41) with eltype Float64 with indices 0:1×0:41:
 0.00253623  6.97666e-9  4.61579e-8  1.44051e-7  …  0.0          1.90665e-28
 0.0         1.16482e-7  4.31593e-6  2.70951e-5     1.22398e-29  0.0

In [30]:
BpAlignGpu.decodeposterior(P, seq.strseq)

("LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP", "LPLGWTEHMSPT-GQPYYFNTQTQESTYVRP", 6, 35)

In [49]:
@time BpAlignGpu.update_F!(af, pm, pa)
@time BpAlignGpu.update_hF!(af, pm, pa)
@time BpAlignGpu.update_B!(af, pm, pa)
@time BpAlignGpu.update_hB!(af, pm, pa)
@time BpAlignGpu.update_beliefs!(af, pm)
@time BpAlignGpu.update_jointchain!(af, pm)
@time BpAlignGpu.update_conditional_chain!(af, pa)
@time BpAlignGpu.update_conditional_all!(af, pm)
@time BpAlignGpu.update_f!(af)
@time BpAlignGpu.update_g!(af)

  0.023638 seconds (348 allocations: 19.422 KiB)
  0.001042 seconds (373 allocations: 20.625 KiB)
  0.000896 seconds (348 allocations: 19.531 KiB)
  0.000842 seconds (419 allocations: 23.688 KiB)
  0.000542 seconds (176 allocations: 10.422 KiB)
  0.000928 seconds (249 allocations: 16.641 KiB)
  0.000694 seconds (342 allocations: 19.688 KiB)
  0.051701 seconds (39.15 k allocations: 2.164 MiB)
  0.098663 seconds (302 allocations: 14.922 KiB)
  0.080164 seconds (323 allocations: 16.828 KiB)


In [50]:
BpAlignGpu.test_sweep!(1,af,pm,pa)

t=1	 err=0.021609247
